<h3> Adapting the 'merge' model to Gray Scale

In [1]:
# Right Train
rightimg_train_gray = []
for i in range(x_train.shape[0]):
    img = image.load_img('Training Images/'+x_train['num'].iloc[i]+'_right.jpg',color_mode="grayscale",target_size=(90,120))
    img = image.img_to_array(img)
    img = img/255
    rightimg_train_gray.append(img)
x_train_r_g = np.array(rightimg_train_gray)

#Left Train
leftimg_train_gray = []
for i in range(x_train.shape[0]):
    img = image.load_img('Training Images/'+x_train['num'].iloc[i]+'_left.jpg',color_mode="grayscale",target_size=(90,120))
    img = image.img_to_array(img)
    img = img/255
    leftimg_train_gray.append(img)
x_train_l_g = np.array(leftimg_train_gray)

# Right Test
rightimg_test_gray = []
for i in range(x_test.shape[0]):
    img = image.load_img('Training Images/'+x_test['num'].iloc[i]+'_right.jpg',color_mode="grayscale",target_size=(90,120))
    img = image.img_to_array(img)
    img = img/255
    rightimg_test_gray.append(img)
x_test_r_g = np.array(rightimg_test_gray)

# Left Test
leftimg_test_gray = []
for i in range(x_test.shape[0]):
    img = image.load_img('Training Images/'+x_test['num'].iloc[i]+'_left.jpg',color_mode="grayscale",target_size=(90,120))
    img = image.img_to_array(img)
    img = img/255
    leftimg_test_gray.append(img)
x_test_l_g = np.array(leftimg_test_gray)

print('Dimensions:')
print(f'Right Train:{x_train_r_g.shape}')
print(f'Left Train:{x_train_l_g.shape}')
print(f'Right Test:{x_test_r_g.shape}')
print(f'Right Test:{x_test_l_g.shape}')

NameError: name 'x_train' is not defined

## Model 2

In [ ]:
left_g = Input(shape=(90,120,1))
conv1 = Conv2D(32, kernel_size=3, activation='relu')(left_g)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
conv1_2 = Conv2D(16, kernel_size=3, activation='relu')(pool1)
pool1_2 = MaxPooling2D(pool_size=(2, 2))(conv1_2)
flat1 = Flatten()(pool1_2)

right_g = Input(shape=(90,120,1))
conv2 = Conv2D(32, kernel_size=3, activation='relu')(right_g)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
conv2_2 = Conv2D(16, kernel_size=3, activation='relu')(pool2)
pool2_2 = MaxPooling2D(pool_size=(2, 2))(conv2_2)
flat2 = Flatten()(pool2_2)
merge = concatenate([flat1, flat2])

In [ ]:
layer1 = Dense(64, activation='relu')(merge)
layer2 = Dense(64, activation='sigmoid')(layer1)
output = Dense(8, activation='sigmoid')(layer2)
model_gray = Model(inputs=[left_g, right_g], outputs=output)
model_gray.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_gray = model_gray.fit(x=[x_train_r_g, x_train_l_g], y=y_train,
                      batch_size=20, epochs=30,validation_split=.1)
model_gray.evaluate([x_test_l_g, x_test_r_g], y_test)

In [ ]:
score = model_gray.evaluate([x_test_l_g, x_test_r_g], y_test, verbose =0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

<h3> Visualize Feature Maps

In [ ]:
# Use the model created
inp = model_c.inputs
print(inp)

# Use the layer where you want to see the convoluted
out1 = model_c.layers[7].output
print(out1)

feature_map_2= Model(inputs=inp, outputs = out1)

feature_map_2.summary()

In [ ]:
# Process one image to see an example
img=cv2.imread("Training Images/0_right.jpg")                   # Reading an Image
img=cv2.resize(img,(100,100))               # Resizing an Image
input_img= np.expand_dims(img, axis=0)      # Expanding the dimension
print(input_img.shape)
f1=feature_map_2.predict(input_img)        # predicting out the Image 
# print(f1.shape)
#                            # Let's see the shape

fig= plt.figure(figsize=(50,50))
for i in range(32):
    ax=fig.add_subplot(8,4,i+1)
    ax.imshow(f1[0,:,:,i])

<h3>Visualize Filters

In [ ]:
# Use your model to create the layers
layers = model_g.layers
layers[7]

#Choose a layer to find a specific set of filters
layer_3 = layers[3]
filter_3, bias_3 = layer_3.get_weights()
print(filter_3.shape ,bias_3.shape)

# Normalize the weights
f_min, f_max = filter_3.min(), filter_3.max()
filter_3 = (filter_3 - f_min) / (f_max - f_min)

fig = plt.figure(figsize = (50,50))
for i in range(32):
    ax = fig.add_subplot(8,4,i+1)
    ax.imshow(filter_3[:,:,:,i])

<h3> Use the new data set with dummy hot encoding

In [ ]:
all_images_gray = []
def get_pixels(values):
    im = Image.open(f'Images//{values}')
        # Adding this to get the gray image
    gray = image.load_img(f'Images//{values}',target_size=(128,128),color_mode="grayscale")
    name = values[:-4]
    im.thumbnail((128,128))
    all_images.append(im)
    all_images_gray.append(gray)
    if name[-5] == 'r':
        all_images_right.append(im)
    else:
        all_images_left.append(im)
    return im

df['pixel_data'] = df['filename'].apply(lambda x: get_pixels(x))

In [ ]:
for x in range(len(all_images_gray)):
    all_images_gray[x] = image.img_to_array(all_images_gray[x])
    all_images_gray[x] = all_images_gray[x]/255
image_array_gray = np.array(all_images_gray)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(image_array_gray, dummy_y, test_size=0.25)

## Model 4

In [ ]:
model = Sequential()
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(128,128,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))  
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(8, activation='sigmoid'))
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train, epochs=10, batch_size=20)

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])